In [12]:
import numpy as np
import sys
if "../" not in sys.path:
    sys.path.append("../")
from collections import defaultdict
from lib import plotting
import matplotlib
from lib.envs.windy_gridworld import WindyGridworldEnv
from lib.envs.random_walk import RandomWalk
import matplotlib.pyplot as plt
from tqdm import tqdm

import itertools
matplotlib.style.use('ggplot')


In [4]:
env2= RandomWalk()

In [8]:
def random_policy(V):
    A = np.ones(env2.action_space.n, dtype=float)/env2.action_space.n
    def policy(state):
        return A
    return policy

In [18]:
def ntd(env, num_episodes, n, discount=1.0, alpha=0.4):
    V = defaultdict(float)
    policy = random_policy(V)
    
    
    for ep in range(num_episodes):
        ret = 0.0
        episode = []
        T = float('inf')
        state = env.reset()
        for t in itertools.count():
            if t < T:
                action = np.random.choice(np.arange(env.action_space.n), p=policy(state))
                next_state, reward, done, _ = env.step(action)
                episode.append((state, action, reward, next_state))
                state = next_state
                if done:
                    T = t+1
            tau = t - n + 1
            if tau >= 0:
                for i in range(tau, min(tau+n, T)):
                    ret += (discount**i * episode[i][2])
                if tau + n < T:
                    ret += (discount**n * V[episode[tau+n-1][3]])
                V[episode[tau][0]] = V[episode[tau][0]] + alpha*(ret - V[episode[tau][0]])
            if tau == T-1:
                break
    for k in V.keys():
        V[k] /= num_episodes
    return V

In [19]:
V = ntd(env2, num_episodes=10, n=4)

In [20]:
print(V)

defaultdict(<class 'float'>, {12: 7.740869868624957e+32, 10: 1.865647290025596e+30, 13: 6.761406062653156e+33, 11: 8.615708110300422e+30, 14: 1.2329510089457664e+34, 15: 1.1808861704738464e+34, 16: 1.0640724400472462e+34, 17: 1.0592707461991257e+34, 18: 6.569897942142563e+33, 19: 6.564357740953107e+33, 9: 1.6005933394809075e+30, 8: 1.1392248972963058e+30, 7: 4.297055286924308e+28, 6: 6.04312672611045e+28, 5: 6.989483794705271e+28, 4: 7.2033460006901305e+28, 3: 5.036989281739275e+28, 2: 9.239782651146545e+27, 1: 2.2872412037760312e+18})
